# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kodinsk,58.6881,99.1844,62.38,55,77,4.07,RU,1628352766
1,1,hithadhoo,-0.6000,73.0833,82.44,67,100,4.47,MV,1628352766
2,2,upernavik,72.7868,-56.1549,41.14,93,100,9.22,GL,1628352767
3,3,bismarck,46.8083,-100.7837,73.87,65,20,3.44,US,1628352602
4,4,avarua,-21.2078,-159.7750,73.45,83,20,4.61,CK,1628352768


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
!pip install gmaps


In [4]:
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [5]:
# Configure  gmaps
gmaps.configure(api_key=g_key)

In [6]:
# heatmap of humidity 
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# Narrowing down cities that fit vriteria and drop any results with null values
narrowed_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) \
                                   & (city_data_df["Wind Speed"] < 10) \
                                   & (city_data_df["Cloudiness"] == 0)].dropna()
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
58,58,drochia,48.0350,27.8161,75.00,48,0,6.51,MD,1628352788
253,253,senanga,-16.1167,23.2667,78.24,11,0,6.20,ZM,1628352863
284,284,sfantu gheorghe,45.8667,25.7833,75.81,50,0,4.94,RO,1628352647
314,314,jieshou,33.2634,115.3611,76.28,95,0,5.26,CN,1628352895
356,356,marovoay,-16.1000,46.6333,75.34,46,0,6.17,MG,1628352915
395,395,pandamatenga,-18.5278,25.6270,72.48,14,0,4.03,BW,1628352930
456,456,binga,-17.6203,27.3414,72.73,32,0,6.64,ZW,1628352957
493,493,tsaratanana,-16.7833,47.6500,71.22,49,0,6.89,MG,1628352973
537,537,zhoukou,33.6333,114.6333,76.86,91,0,5.93,CN,1628352991
543,543,fuyang,32.9000,115.8167,75.29,92,0,5.88,CN,1628352994


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# Create a Dataframe called hotel_df to store hotel named with city
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
58,drochia,MD,48.0350,27.8161,
253,senanga,ZM,-16.1167,23.2667,
284,sfantu gheorghe,RO,45.8667,25.7833,
314,jieshou,CN,33.2634,115.3611,
356,marovoay,MG,-16.1000,46.6333,
395,pandamatenga,BW,-18.5278,25.6270,
456,binga,ZW,-17.6203,27.3414,
493,tsaratanana,MG,-16.7833,47.6500,
537,zhoukou,CN,33.6333,114.6333,
543,fuyang,CN,32.9000,115.8167,


In [9]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    #get lat and long
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    #use the search term: hotel and out lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    name_address = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except(KeyError, IndexError):
        print("Missing field/result...skipping")
hotel_df

Missing field/result...skipping
Missing field/result...skipping
Missing field/result...skipping
Missing field/result...skipping
Missing field/result...skipping
Missing field/result...skipping
Missing field/result...skipping
Missing field/result...skipping
Missing field/result...skipping
Missing field/result...skipping


,City,Country,Lat,Lng,Hotel Name
58,drochia,MD,48.0350,27.8161,
253,senanga,ZM,-16.1167,23.2667,
284,sfantu gheorghe,RO,45.8667,25.7833,
314,jieshou,CN,33.2634,115.3611,
356,marovoay,MG,-16.1000,46.6333,
395,pandamatenga,BW,-18.5278,25.6270,
456,binga,ZW,-17.6203,27.3414,
493,tsaratanana,MG,-16.7833,47.6500,
537,zhoukou,CN,33.6333,114.6333,
543,fuyang,CN,32.9000,115.8167,


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))